In [3]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import GRFski as gski

ModuleNotFoundError: No module named 'gstools'

## Example using only one realisation of the GRF and plot it

In [ ]:
H_trans,p_grid,E_slab,thau,Tp,Ac = gski.gen_GRF2d_arr_pslab(10,0.8,0.0075,20)
lsk_grid,Sp_grid = gski.lsk_ongrid2(H_trans,thau,Tp,Ac)
skieur_grid,proba_trigger = gski.skieur_proba(Sp_grid,20,10,10)

plt.rcParams["font.family"] = "serif"
fig,((ax1,ax2),(ax3,ax4),(ax5,ax6)) = plt.subplots(ncols = 2,nrows = 3, figsize = (9,7),sharex=True, sharey=True, constrained_layout = True)
fig.supxlabel("Distance cross-slope (m)", fontsize = 18)
fig.supylabel("Distance upslope (m)", fontsize = 18)
#fig.suptitle('Numerical simulation of skier triggering on a heterogenous snowpack', fontsize = 18)

p1 = ax1.imshow(H_trans, interpolation='none',cmap = "viridis", extent = [0,100,0,50])
plt.colorbar(p1,ax=ax1)
ax1.set_title("Slab depth (m)", fontsize = 14)

p2 = ax2.imshow(Tp, interpolation='none', cmap = "viridis", extent = [0,100,0,50])
plt.colorbar(p2,ax=ax2)
ax2.set_title("Shear strength (Pa)", fontsize = 14)

p3 = ax3.imshow(lsk_grid, interpolation='none', cmap = "viridis", extent = [0,100,0,50])
plt.colorbar(p3,ax=ax3)
ax3.set_title("Skier crack length $l_{sk}$ (m)", fontsize = 14)

p4 = ax4.imshow(Ac, interpolation='none', cmap = "viridis", extent = [0,100,0,50])
plt.colorbar(p4,ax=ax4)
ax4.set_title("Critical crack length $A_c$ (m)", fontsize = 14)

p5 = ax5.imshow(Sp_grid, interpolation='none', cmap = "Spectral", extent = [0,100,0,50])
plt.colorbar(p5,ax=ax5)
ax5.set_title("Skier propagation index SPI", fontsize = 14)

p6 = ax6.imshow(skieur_grid, interpolation='none', extent = [0,100,0,50])
plt.colorbar(p6,ax=ax6)
ax6.set_title("Skier tracks with SPI", fontsize = 14)


#plt.savefig("GRF_real.eps", dpi=100)


## Create a list of the parameter we need to test 
    1) Mean slab depth
    2) Correlation length
    3) Slab depth variance
    4) Space between skier
    5) Skier radius

In [6]:
mean_values = [0.7]
var_values = [0.0025]
lenscale_values = [5,10,15,20,25,30]
space_values = [10]
radius_values = [5,10,15,20,25,30,35,40]
ens_set2 = []
for m in mean_values:
    for v in var_values:
        for l in lenscale_values:
            for s in space_values:
                for r in radius_values:
                    l_radius_ratio = l/r
                    l_space_ratio = l/s
                    print("mean:", m, " variance:", v, " correlation length:",l, " radius skieur:", r, " space_skieur:",s)
                    set_var = [m,v,l,r,s,l_radius_ratio, l_space_ratio]
                    ens_set2.append(set_var)
print(len(ens_set2))

mean: 0.7  variance: 0.0025  correlation length: 5  radius skieur: 5  space_skieur: 10
mean: 0.7  variance: 0.0025  correlation length: 5  radius skieur: 10  space_skieur: 10
mean: 0.7  variance: 0.0025  correlation length: 5  radius skieur: 15  space_skieur: 10
mean: 0.7  variance: 0.0025  correlation length: 5  radius skieur: 20  space_skieur: 10
mean: 0.7  variance: 0.0025  correlation length: 5  radius skieur: 25  space_skieur: 10
mean: 0.7  variance: 0.0025  correlation length: 5  radius skieur: 30  space_skieur: 10
mean: 0.7  variance: 0.0025  correlation length: 5  radius skieur: 35  space_skieur: 10
mean: 0.7  variance: 0.0025  correlation length: 5  radius skieur: 40  space_skieur: 10
mean: 0.7  variance: 0.0025  correlation length: 10  radius skieur: 5  space_skieur: 10
mean: 0.7  variance: 0.0025  correlation length: 10  radius skieur: 10  space_skieur: 10
mean: 0.7  variance: 0.0025  correlation length: 10  radius skieur: 15  space_skieur: 10
mean: 0.7  variance: 0.0025  co

# Run the skier simulation with 10 realisations 
Ideally 100 is better but it depends on computing time

In [ ]:
%%time
distribution_probaGRF,m_std_probaGRF = gski.gen_probaskieur_multireal(ens_set2,10)

In [ ]:
#Convert to a pandas dataframe
mstd_proba_PD = pd.DataFrame.from_dict(m_std_probaGRF, orient = "index",
                                       columns = ["mean_slab","var_slab","lscale_slab","Rayon_skieur","Space_skieur","Nb_skieur","l_rayon_ratio","l_space_ratio","mean_proba","std_proba"])
mstd_proba_PD

# Adding skier from a safe track until trigger
    1) Random
    2) Structure

In [ ]:
%%time
H_trans,p_grid,E_slab,thau,Tp,Ac = gski.gen_GRF2d_arr_pslab(4,0.9,0.0075,10)
lsk_grid,Sp_grid = gski.lsk_ongrid2(H_trans,thau,Tp,Ac)

random_position_sk1 = np.random.uniform(50,1900)
print(random_position_sk1)
skieur_grid_struct,nb_skieur = gski.skieur_addgroup_struct(Sp_grid,5,10,random_position_sk1)
skieur_grid_alea, nbskier_alea = gski.skieur_addgroup_alea(Sp_grid,5,10,random_position_sk1)

plt.imshow(H_trans, interpolation='none')
plt.colorbar()
plt.show()

plt.imshow(Sp_grid, interpolation='none', cmap = "Spectral")
plt.colorbar()
plt.clim(0,2)
plt.show()

#plt.imshow(skieur_grid_struct, interpolation='none')
#plt.colorbar()
#plt.clim(0,2)
#plt.show

plt.imshow(skieur_grid_alea, interpolation='none')
plt.colorbar()
plt.clim(0,2)
plt.show

print(nb_skieur)
print(nbskier_alea)

plt.rcParams["font.family"] = "serif"
fig,(ax1,ax2) = plt.subplots(ncols = 2,nrows = 1, figsize = (13,4),sharex=True, sharey=True, constrained_layout = True)
fig.supxlabel(" distance cross-slope (m)", fontsize = 18)
fig.supylabel("distance up-slope (m)", fontsize =18)

p1 = ax1.imshow(skieur_grid_struct, interpolation='none',cmap = "viridis", extent = [0,200,0,100])
#cbar 1 = plt.colorbar(p1,ax=ax1)
#ax1.set_title("Structured additional skier", fontsize = 20)
ax1.tick_params(axis='both', which='major', labelsize=14)


p2 = ax2.imshow(skieur_grid_alea, interpolation='none',cmap = "viridis", extent = [0,200,0,100])
cbar = plt.colorbar(p2,ax=ax2)
#ax2.set_title("Random additional skier", fontsize = 20)
ax2.tick_params(axis='both', which='major', labelsize=14)

cbar.ax.tick_params(labelsize=14)

fig.savefig('random_additional_skier.jpg', dpi=300)

## Example for correlation length of 20 m

In [ ]:
%%time

# Example for a corrleation length 0f 20
nbskier_struct_dict_20 = {}
nbskier_alea_dict_20 = {}

lscale = 20
meanslab = 0.7
varslab = 0.005
space_skier = 5
radius_skier = 10
for real in range(0,100,1):
    H_trans,p_grid,E_slab,thau,Tp,Ac = gski.gen_GRF2d_arr_pslab(real,meanslab,varslab,lscale)

    nbskier_struct_list = []
    nbskier_alea_list = []
    for i in range(0,100,1):
        random_position_sk1 = np.random.uniform(50,1900)
        lsk_grid,Sp_grid = gski.lsk_ongrid2(H_trans,thau,Tp,Ac)

        skieur_grid_struct, nbskier_struct = gski.skieur_addgroup_struct(Sp_grid,space_skier,radius_skier,random_position_sk1)
        if nbskier_struct in range(1,49,1): # stop the process after 50 skier
            nbskier_struct_list.append(nbskier_struct)


        skieur_grid_alea, nbskier_alea = gski.skieur_addgroup_alea(Sp_grid,space_skier,radius_skier,random_position_sk1)
        if nbskier_alea in range(1,49,1): # stop the process after 50 skier
            nbskier_alea_list.append(nbskier_alea)

    nbskier_struct_dict_20.update({str(real):np.array(nbskier_struct_list)})
    nbskier_alea_dict_20.update({str(real):np.array(nbskier_alea_list)})
  